<img width="20%" alt="EarthDaily Agro" src="https://earthdaily.com/hubfs/EDA_logo_horizontal_gradient-navy_1.svg" style="border-radius: 12px" />

# Use Case Analytics

Download artifacts for a use case, enrich them with analytics APIs, and publish a curated version.

**Notebook map**
- 🔐 Setup & fetch artifacts
- 🌱 Emergence analytics via `run_task`
- 🌦️ Weather analytics with resumable checkpoints
- 📤 Export the enriched dataset


## Before you start

- `.env` (shared with the other notebooks) must contain API credentials plus `EMERGENCE`/`WEATHER` (etc) endpoints.
- Keep a local copy of the enriched parquet from Notebook 2 (client_use_case) handy to skip re-downloading artifacts when rehearsing.
- Each analytics section exposes the parameters you are most likely to tweak during demos (crop defaults, worker counts, checkpoint cadence, etc.).


In [3]:
# --- bootstrap, environment, and shared clients ----------------------------------
import logging
import sys
from pathlib import Path

from IPython.display import display, HTML, Markdown


logging.basicConfig(level=logging.INFO)

import pandas as pd
from dotenv import load_dotenv


# Make project importable
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
import requests
from lib import ApiSession, download_parquet_to_df, load_local_parquet, publish_group_artifact
from lib.domain.crop_cycle import CropCycleCatalog


# Authentication context (.env is shared with the other notebooks)
env_path = Path('..') / '.env'
if env_path.exists():
    load_dotenv(env_path)


session = ApiSession.from_env()
session
from lib.analytics import (
    BindingKey,
    EmergenceParams,
    format_bindings_catalog_html,
    GreennessDetectionParams,
    InseasonPotentialScoreParams,
    LriParams,
    run_task,
    run_task_with_reporting,
    render_final_report,
    apply_notebook_style,
    show_task_summary,
    AnalyticsRunReporter,
    WeatherParams,
)
apply_notebook_style()

## Quick navigation
- [Select use case](#1.-Select-a-use-case-and-fetch-the-latest-artifact-metadata)
- [Preview parquet](#2.-Preview-parquet)
- [Crop mask recap](#3.-(Optional)-Crop-mask-recap)
- [Local parquet option](#4.-(Optional)-Load-a-local-enriched-parquet)
- [Emergence](#5.-Emergence-processor-enrichment)
- [Weather](#6.-Weather-analytics)
- [Greenness detection](#Greenness-detection)
- [Inseason potential score](#Inseason-potential-score)
- [LRI NDVI](#7.-LRI-NDVI-accumulation)
- [Export & publish](#8.-Export-and-publish-a-new-artifact-version)


### Logging controls

This cell configures logging for the analytics below. All loggers start at INFO; bump a specific task to DEBUG or lower it to WARNING as needed.


In [ ]:
# Notebook logging: set analytics loggers to INFO by default.
logging.getLogger().setLevel(logging.INFO)
logging.getLogger("lib.analytics").setLevel(logging.INFO)
logging.getLogger("analytics.emergence").setLevel(logging.INFO)
logging.getLogger("analytics.weather").setLevel(logging.INFO)
logging.getLogger("analytics.greenness_detection").setLevel(logging.INFO)
logging.getLogger("analytics.inseason_potential_score").setLevel(logging.INFO)
logging.getLogger("analytics.lri").setLevel(logging.INFO)

# Example: raise or lower verbosity per task (e.g., DEBUG for troubleshooting)
# logging.getLogger("analytics.emergence").setLevel(logging.DEBUG)


## Available analytics bindings


In [ ]:
# Analytics tasks information (parameters to bind)
HTML(format_bindings_catalog_html())

## 1. Select a use case and fetch the latest artifact metadata

Provide `group_id` (use case).


In [ ]:
# Provide a group_id
group_id = ''

# Reuse the group we just provisioned; override `group_id` above if you want to inspect a different use case.
artifacts = session.get(f'/v1/groups/{group_id}/artifacts')
if not artifacts:
    raise RuntimeError('No artifacts available for this group yet.')
latest_version = artifacts[0]['version']
parquet_url = session.get(f'/v1/groups/{group_id}/presign', params={'artifact_type': 'parquet', 'version': latest_version})['url']
manifest_url = session.get(f'/v1/groups/{group_id}/presign', params={'artifact_type': 'manifest', 'version': latest_version})['url']

parquet_df = download_parquet_to_df(parquet_url)
manifest = requests.get(manifest_url, timeout=60).json()

# Keep an untouched copy for alternate enrichment flows
parquet_df_original = parquet_df.copy()

desc = manifest.get('description') or 'No description provided.'
schema_df = pd.DataFrame(manifest.get('schema') or [])
row_count = manifest.get('row_count') or ""
file_path = manifest.get('artifacts', '').get('parquet', '')
file_name = str(file_path).split('/')[-1]

display(Markdown(f"**Latest version:** v{latest_version}"))
display(Markdown(f"**Manifest description:** {desc}"))
display(Markdown(f"**Number of elements:** {row_count}"))
display(Markdown(f"**Parquet file name:** {file_name}"))

if not schema_df.empty:
    display(Markdown('**Manifest schema (preview)**'))
    display(schema_df)
else:
    display(Markdown('_No schema provided in manifest._'))

pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)

### Execution report (HTML + summary)
Enable the built-in reporter to collect task stats and generate an HTML recap at the end of the notebook.

In [ ]:
from lib.analytics import AnalyticsRunReporter

reporter = AnalyticsRunReporter(
    enabled=True,
    html_path="tmp/analytics_run_report.html",
    include_map=True,
    map_geometry_column="geometry",
    map_max_features=800,
    map_simplify_tolerance=0.0005,
)

reporter.set_run_context(
    group_id=group_id,
    parquet_version=latest_version,
    manifest=manifest,
    parquet_path=file_path,
)
reporter.capture_dataset(parquet_df, sample_rows=50, use_for_map=True)


## 2. Preview parquet


In [ ]:
display(Markdown(f"Parquet rows: **{len(parquet_df)}**"))
parquet_df.head()

## 3. (Optional) Crop mask recap

This use case assumes the parquet already contains the current crop information gathered during initialization. Keep this section placeholder if you later decide to re-run crop filtering here.


## 4. (Optional) Load a local enriched parquet

Use the file exported by the client-use-case notebook to skip API latency. Toggle the flag in the next cell to keep the freshly downloaded artifact instead.

| Switch | Meaning |
| --- | --- |
| `USE_LOCAL_PARQUET` | `True` → open the local file, `False` → keep the freshly downloaded artifact |
| `LOCAL_PARQUET_FILENAME` | Update with the file you exported in Notebook 2 |

reporter.capture_dataset(parquet_df, use_for_map=True)


In [ ]:
USE_LOCAL_PARQUET = True  # flip to False to rely on the downloaded artifact
LOCAL_PARQUET_FILENAME = 'your_file.parquet'

if USE_LOCAL_PARQUET:
    # local_parquet_path = Path('..') / '2-client_use_case' / 'local_exports' / LOCAL_PARQUET_FILENAME
    local_parquet_path = Path('.') / 'local_exports' / LOCAL_PARQUET_FILENAME
    if not local_parquet_path.exists():
        raise FileNotFoundError(f"Local parquet not found: {local_parquet_path}")
    parquet_df = load_local_parquet(local_parquet_path)
    print(f"Loaded {len(parquet_df)} rows from {local_parquet_path}")
else:
    print('Skipping local parquet override; using the artifact downloaded above.')

# parquet_df=parquet_df[0:10]
reporter.capture_dataset(parquet_df, use_for_map=True)
parquet_df

### Note on **task_params** vs **defaults_override**
`task_params` drives the task logic (validation, internal options). `defaults_override` fills input columns when they are missing or null in the parquet (e.g., `year`, `data_source`).

Emergence example:
- `EmergenceParams(year=2024, data_source='LR', ...)` ? passed to the task logic.
- `defaults_override={'year': 2024}` ? injects the value into the `year` column if absent in the parquet.

Tip: keep a single `year` variable and reuse it in both `EmergenceParams` **and** `defaults_override` to avoid ambiguity.


## 5. Emergence processor enrichment

This section augments the artifact with emergence analytics using `run_task(binding_key="emergence")`.

1. Load the crop-cycle catalog from `config/` (state/county overrides kick in automatically when available).
2. Configure the base parameters (target year, optional season overrides, data source, concurrency).
3. Let `run_task` normalize the dataframe, call the Emergence API, and merge the results back while preserving existing columns.

| Parameter | Description |
| --- | --- |
| `default_crop` | Crop fallback when the parquet lacks a `crop_code` |
| `season_*` | Manual overrides (keep `None` to rely on catalog defaults) |
| `max_workers` | Thread pool size for Emergence API calls |
| `checkpoint_path` | Enable persistent checkpoints if you want resumable runs |


In [ ]:
HTML(format_bindings_catalog_html(binding_key=BindingKey.EMERGENCE))

In [ ]:
config_dir = Path('..') / 'config'
crop_cycles = CropCycleCatalog.from_csv(
    path=config_dir / 'crop_cycle.csv',
    state_timewindows_path=config_dir / 'state_level_timewindows.csv',
    county_timewindows_path=config_dir / 'county_level_timewindows.csv',
    )
print(f"Crop-cycle configuration loaded from {config_dir.resolve()}")


emergence_params_model = EmergenceParams(
    season_duration=None,
    season_start_day=None,
    season_start_month=None,
    year=2025,
    data_source='LR',
    default_crop='CORN',
    max_workers=25,
    checkpoint_path='tmp/emergence_checkpoint.parquet',
    checkpoint_frequency=500,
    )
emergence_params = emergence_params_model.model_dump(exclude_none=False)

parquet_df, emergence_stats = run_task_with_reporting(
    parquet_df,
    binding_key=BindingKey.EMERGENCE,
    defaults_override={'year': emergence_params['year']},
    task_params={**emergence_params, 'crop_cycles': crop_cycles, 'region_column': 'n_county'},
    progress=True,
    progress_desc="Emergence",
    error_csv_path="tmp/errors/emergence_errors.csv",
    checkpoint_path=emergence_params.get('checkpoint_path'),
    checkpoint_frequency=emergence_params.get('checkpoint_frequency', 0),
    reporter=reporter,
    )


parquet_df

**Tip:** pass `error_csv_path=\"tmp/errors/emergence_errors.csv\"` to `run_task` to export rows with error columns when failures occur. The file is overwritten each run (directories auto-created).

## 6. Weather analytics

Derive planting dates (when missing), resolve tasseling/R3/black-layer offsets per county from the agronomist config, and call the Weather API per geohash. Checkpoints are written to `tmp/weather_checkpoint.parquet` and now include the **full dataframe merged with partial results**, so you can resume by simply reloading the file.

| Parameter | Description |
| --- | --- |
| `WEATHER_PROVIDER` | API provider (e.g., `global1`, `DTN`) |
| `WEATHER_CONTEXT_DAYS` | Size of rainfall context windows (past/future) |
| `WEATHER_MAX_WORKERS` | Parallel workers per weather region |
| `DEFAULT_WEATHER_REGION` | Fallback when `n_county` cannot be resolved |
| `WEATHER_CHECKPOINT` | Path to the auto-merged checkpoint parquet |


In [ ]:
HTML(format_bindings_catalog_html(binding_key=BindingKey.WEATHER))

### Input rules (optional)

Use `input_rules` to skip rows before running analytics:
- Each rule targets one column and can define sentinels to skip (`invalid_values`) and whether to count the row as skipped (`skip_row=True`).
- Skipped rows are not processed by the analytics task but remain in the final dataframe; they are counted in `TaskStats.filtered_out`.
- Keep rules minimal to avoid changing signatures unexpectedly; prefer status/sentinel checks over heavy date coercion when possible.

Example used here:
input_rules = [
    {"column": "emergence_confirmation_status", "invalid_values": ["no_emergence"], "skip_row": True},
]
Pass them directly to `run_task(..., input_rules=input_rules)`.


In [ ]:
input_rules = [{"column": "emergence_confirmation_status", "invalid_values": ["no_emergence"], "skip_row": True}]  # optional

In [ ]:
# --- Weather analytics configuration -------------------------------------------------
from pathlib import Path


weather_params = WeatherParams(
    provider='global1',
    lower_threshold=50,
    upper_threshold=86,
    rainfall_context_days=10,
    max_workers=6,
    default_region='Dodge',
    checkpoint_path='tmp/weather_checkpoint.parquet',
    checkpoint_frequency=500,
    )

parquet_df, weather_stats = run_task_with_reporting(
    parquet_df,
    binding_key=BindingKey.WEATHER,
    input_rules=input_rules,
    error_csv_path="tmp/errors/weather_errors.csv",
    task_params=weather_params.model_dump(exclude_none=False),
    checkpoint_path=weather_params.checkpoint_path,
    checkpoint_frequency=weather_params.checkpoint_frequency,
    progress=True,
    progress_desc="Weather",
    reporter=reporter,
    )

pd.set_option('display.max_columns', None)
parquet_df

**Tip:** pass `error_csv_path=\"tmp/errors/weather_errors.csv\"` to `run_task` to export rows with error columns when failures occur. The file is overwritten each run (directories auto-created).

## Greenness detection

In [ ]:
HTML(format_bindings_catalog_html(binding_key=BindingKey.GREENNESS_DETECTION))

In [ ]:
config_dir = Path('..') / 'config'
crop_cycles = CropCycleCatalog.from_csv(
    path=config_dir / 'crop_cycle.csv',
    state_timewindows_path=None,
    county_timewindows_path=None,
    )
print(f"Crop-cycle configuration loaded from {config_dir.resolve()}")

greenness_params_model = GreennessDetectionParams(
    seasonDuration=None,
    seasonStartDay=None,
    seasonStartMonth=None,
    year=2025,
    dataSource='LR',
    default_crop='CORN',
    max_workers=25,
    checkpoint_path='tmp/greenness_checkpoint.parquet',
    checkpoint_frequency=500,
    )
greenness_params = greenness_params_model.model_dump(exclude_none=False)

parquet_df, greenness_stats = run_task_with_reporting(
    parquet_df,
    binding_key=BindingKey.GREENNESS_DETECTION,
    input_rules=input_rules,
    field_mapping_override={"sowingDate": "planting_date", "crop": "current_crop"},
    defaults_override={"year": 2025, "dataSource": "LR"},  # constant values
    task_params={**greenness_params, "crop_cycles": crop_cycles, "region_column": "n_county"},
    progress=True,
    progress_desc="Greenness",
    error_csv_path="tmp/errors/greenness_detection_errors.csv",
    checkpoint_path=greenness_params.get("checkpoint_path"),
    checkpoint_frequency=greenness_params.get("checkpoint_frequency", 0),
    reporter=reporter,
    )



parquet_df

**Tip:** pass `error_csv_path=\"tmp/errors/greenness_detection_errors.csv\"` to `run_task` to export rows with error columns when failures occur. The file is overwritten each run (directories auto-created).

## Inseason potential score

In [ ]:
HTML(format_bindings_catalog_html(binding_key=BindingKey.INSEASON_POTENTIAL_SCORE))

In [ ]:
config_dir = Path('..') / 'config'
crop_cycles = CropCycleCatalog.from_csv(
    path=config_dir / 'crop_cycle.csv',
    state_timewindows_path=None,
    county_timewindows_path=None,
    )
print(f"Crop-cycle configuration loaded from {config_dir.resolve()}")

inseason_potential_score_params_model = InseasonPotentialScoreParams(
    seasonDuration=None,
    seasonStartDay=None,
    seasonStartMonth=None,
    threshold=0.67,
    year=2025,
    dataSource='LR',
    default_crop='CORN',
    max_workers=25,
    checkpoint_path='tmp/inseason_potential_score_checkpoint.parquet',
    checkpoint_frequency=500,
    numberHistoricalYears=5,
    )
inseason_potential_score_params = inseason_potential_score_params_model.model_dump(exclude_none=False)

parquet_df, inseason_potential_score_stats = run_task_with_reporting(
    parquet_df,
    binding_key=BindingKey.INSEASON_POTENTIAL_SCORE,
    input_rules=input_rules,
    field_mapping_override={
        "sowingDate": "planting_date",
        "crop": "current_crop",
        "historicalSeasons": "crop_history_years_list",
    },  # selon tes colonnes
    defaults_override={"year": 2025, "dataSource": "LR", "numberHistoricalYears": 5, "threshold": 0.67},  # valeurs fixes
    task_params={**inseason_potential_score_params, "crop_cycles": crop_cycles, "region_column": "n_county"},
    progress=True,
    progress_desc="Inseason Potential Score",
    error_csv_path="tmp/errors/inseason_potential_score_errors.csv",
    checkpoint_path=inseason_potential_score_params.get("checkpoint_path"),
    checkpoint_frequency=inseason_potential_score_params.get("checkpoint_frequency", 0),
    reporter=reporter,
    )



parquet_df

**Tip:** pass `error_csv_path=\"tmp/errors/inseason_potential_score_errors.csv\"` to `run_task` to export rows with error columns when failures occur. The file is overwritten each run (directories auto-created).

## 7. LRI NDVI accumulation

Fetch NDVI time series from LRI for each spatial unit, then compute per-year sliding-window sums.
- Uses `crop_history_years` to derive which seasons to accumulate.
- Window end date defaults to today; override via `end_date` if you need a fixed cutoff.


In [ ]:
HTML(format_bindings_catalog_html(binding_key=BindingKey.LRI_NDVI))

In [ ]:
from datetime import date


pd.set_option('display.max_colwidth', None)

lri_params = LriParams(
    window_days=20,
    end_date=date(2024, 11, 20),
    max_workers=2,
    max_concurrency_cap=32,
    batch_size=25,
    checkpoint_path='tmp/lri_checkpoint.parquet',
    checkpoint_frequency=50,
    )

parquet_df, lri_stats = run_task_with_reporting(
    parquet_df,
    binding_key=BindingKey.LRI_NDVI,
    input_rules=input_rules,
    error_csv_path="tmp/errors/lri_ndvi_errors.csv",
    task_params=lri_params.model_dump(exclude_none=False),
    field_mapping_override={'geometry': 'geometry'},
    progress=True,
    progress_desc='LRI NDVI accumulation',
    trace=True,
    checkpoint_path=lri_params.checkpoint_path,
    checkpoint_frequency=lri_params.checkpoint_frequency,
    reporter=reporter,
    )
parquet_df[['spatial_unit_id', 'lri_accumulation']]

**Tip:** pass `error_csv_path=\"tmp/errors/lri_ndvi_errors.csv\"` to `run_task` to export rows with error columns when failures occur. The file is overwritten each run (directories auto-created).

## 8. Export and publish a new artifact version

Write the enriched dataframe to GeoParquet and publish it back to the group using the presign/confirm helper.

| Parameter | Description |
| --- | --- |
| `group_code` | Use the same group selected above |
| `output_path` | Local parquet path written below |
| `description` | Optional manifest note for provenance |

---


### Execution report
Render the HTML report (written to `tmp/analytics_run_report.html`) and a short summary you can reuse as the artifact description.

In [ ]:
reporter.capture_dataset(parquet_df, use_for_map=True)
report_html = render_final_report(reporter, show_inline=True)
summary_md = reporter.render_summary_markdown()
display(Markdown(summary_md))


### Set artifact description
Update the description before publishing the enriched artifact.


In [ ]:
# Edit the artifact description before publishing Manually:
description = "Demo Test - crop mask (CORN)  + emergence + weather + greenness + inseason potential score + LRI"

# Or automatic description created:
description= reporter.render_summary_markdown()


### Upload the new parquet file version


In [ ]:

filename = 'data.parquet'

local_export_dir = Path('local_exports')
local_export_dir.mkdir(exist_ok=True)
local_baseline_path = local_export_dir / f"{group_id}_enriched_baseline.parquet"

confirmation = publish_group_artifact(
    session,
    group_code=group_id,
    df=parquet_df,
    filename=filename,
    persist_path=local_baseline_path,
    description=description,
)

job_status = confirmation.get('job_status', {})
state = job_status.get('state', 'unknown')
message = job_status.get('message', '')

display(Markdown('---'))
display(Markdown('### Artifact publish result'))
display(Markdown(f'- **State:** `{state}`'))
if message:
    display(Markdown(f'- **Message:** {message}'))
display(Markdown(f'- **Local file:** `{local_baseline_path}`'))
display(Markdown('---'))